# NLP Topic Modeling: Obama VS Trump (Round II)

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

### Import packages

In [1]:
# ! pip install gensim

In [2]:
import pandas as pd
import numpy as np # np.random.seed(2018)
import re

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
# nltk.download('wordnet')

unable to import 'smart_open.gcs', disabling that module


### Import data

In [3]:
# https://www.kaggle.com/austinreese/trump-tweets
df_trump = pd.read_csv('./datasets/trump_tweets.csv')
# https://www.kaggle.com/datacrux/barack-obama-twitterdata-from-20122019
df_obama = pd.read_csv('./datasets/obama_tweets.csv')

### Explore data

In [4]:
df_trump.head(1)

,id,link,content,date,retweets,favorites,mentions,hashtags,geo
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 20:54:25,500,868,NaN,NaN,NaN


In [5]:
df_obama.head(1)

,Date,Username,Tweet-text,Tweet Link,Retweets,Likes,TweetImageUrl,Image
0,2019/04/10_17:08,https://twitter.com/BarackObama,From a big NBA fan congrats to future Hall of ...,https://twitter.com/BarackObama/status/1116130...,20954,175133,NaN,NaN


### Data preprocessing

#### Deleting links

In [6]:
# pic.twitter.com
df_trump['content'] = df_trump['content'].map(lambda x: re.sub('pic.twitter.com/\w\w\w\w\w\w\w\w\w\w', '', x))
df_obama['Tweet-text'] = df_obama['Tweet-text'].map(lambda x: re.sub('pic.twitter.com/\w\w\w\w\w\w\w\w\w\w', '', x))

In [7]:
# tinyurl
df_trump['content'] = df_trump['content'].map(lambda x: re.sub('http://tinyurl.com/\w\w\w\w\w\w', '', x))
df_obama['Tweet-text'] = df_obama['Tweet-text'].map(lambda x: re.sub('http://tinyurl.com/\w\w\w\w\w\w', '', x))

In [8]:
# bit.ly
df_trump['content'] = df_trump['content'].map(lambda x: re.sub('http://bit.ly/\w\w\w\w\w\w', '', x))
df_obama['Tweet-text'] = df_obama['Tweet-text'].map(lambda x: re.sub('http://bit.ly/\w\w\w\w\w\w', '', x))

In [9]:
# Others:
df_trump['content'] = df_trump['content'].map(lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
df_obama['Tweet-text'] = df_obama['Tweet-text'].map(lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))